# Image Arcface Loss Method

## Mount

In [1]:
from google.colab import drive 
drive.mount("/content/drive", force_remount=True)
%cd /content/drive/MyDrive/Colab\ Notebooks/dl-ecommerce-duplicates/
!mkdir /content/data/
!cp /content/drive/MyDrive/Colab\ Notebooks/shopee-product-matching.zip /content/data/shopee-product-matching.zip
!unzip -q /content/data/shopee-product-matching.zip -d /content/data/shopee-product-matching
!cp /content/drive/MyDrive/Colab\ Notebooks/dl-ecommerce-duplicates/data/train80.csv /content/data/shopee-product-matching/train80.csv
!cp /content/drive/MyDrive/Colab\ Notebooks/dl-ecommerce-duplicates/data/valid20.csv /content/data/shopee-product-matching/valid20.csv

Mounted at /content/drive
/content/drive/MyDrive/Colab Notebooks/dl-ecommerce-duplicates


## Imports

In [2]:
!pip install timm
!pip install ray[tune]
!pip install transformers
!apt install libomp-dev
!python -m pip install --upgrade faiss-gpu==1.7.2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 549.1/549.1 kB 993.8 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 26.5 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.6/58.6 MB 29.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 19.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 110.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 106.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 16.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.5/468.5 kB 46.6 MB/s eta 0:00:00
  Attempting uninstall: grpcio
    Found existing installation: grpcio 1.54.0
    Uninstalling grpcio-1.54.0:
      Successfully uninstalled grpcio-1.54.0
L

In [3]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
from warnings import filterwarnings
from sklearn.preprocessing import LabelEncoder
import random
import numpy as np
from tqdm import tqdm
filterwarnings("ignore")

from modules.datasets.ImageArcFaceLossShopeeDataset import ImageArcFaceLossShopeeDataset
from modules.losses.ArcFaceLoss import ArcFaceLoss
from modules.models.ImageShopeeNet import ImageShopeeNet
from modules.distances.CosineDistance import CosineDistance

import modules.utils.dataset_utils as dataset_utils

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("You are using device: %s" % device)

seed = 42
random.seed(seed)
os.environ['PYTHONHASHSEED'] = str(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

DATA_FOLDER = '/content/data/shopee-product-matching/'

You are using device: cuda


## Data

In [4]:
DATA_FOLDER = '/content/data/shopee-product-matching/'

train_df = dataset_utils.get_dataset(DATA_FOLDER, is_test=False, file_name='train80.csv')
train_df = dataset_utils.add_target(train_df)

valid_df = dataset_utils.get_dataset(DATA_FOLDER, is_test=False, file_name='valid20.csv')
valid_df = dataset_utils.add_target(valid_df)

all_df = train_df.append(valid_df, ignore_index=True)

print(f"All shape {all_df.shape}")
print(f"Train shape {train_df.shape}")
print(f"Valid shape {valid_df.shape}")

All shape (34250, 7)
Train shape (27399, 7)
Valid shape (6851, 7)


## Augmentations

In [5]:
image_size = 512

train_transforms = transforms.Compose([
	transforms.Resize((image_size, image_size)),
	transforms.RandomPosterize(bits=2, p=0.3),
	transforms.RandomHorizontalFlip(p=0.4),
	transforms.RandomAutocontrast(p=0.3),
	transforms.ToTensor(),
	transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])
valid_transforms = transforms.Compose([
	transforms.Resize((image_size, image_size)),
	transforms.ToTensor(),
	transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

train_dataset = ImageArcFaceLossShopeeDataset(
	train_df['image'].values, 
	train_df['label_group'].values,
	train_transforms
)
valid_dataset = ImageArcFaceLossShopeeDataset(
	valid_df['image'].values, 
	valid_df['label_group'].values, 
	valid_transforms
)

print(len(train_dataset))
print(len(valid_dataset))

27399
6851


## CFG

In [13]:
class CFG:
    IMG_SZ = 512
    TRAIN_RATIO = 0.8
    EPOCHS = 10
    MARGIN = 0.75
    DISTANCE = CosineDistance()
    image_size = 512
    batch_size = 16
    n_worker = 4
    init_lr = 3e-4
    dropout = 0.5
    n_epochs = 30
    n_splits = 5
    n_samples = None
    fold_id = 0
    valid_every = 1
    save_after = 0
    scale = 10
    margin = 0.5
    easy_margin = False
    search_space = np.arange(40, 100, 10)
    debug = False
    kernel_type = 'baseline'
    model_dir = './weights/'
    data_dir = './data/train_images'
    csv = './data/train.csv'
    n_classes = 11014


## Training

In [14]:
def get_model(config):
    return ImageShopeeNet(
		model_name=config["model_name"],
		fc_dim=config["fc_dim"],
		freeze_layers=config["freeze_layers"], 
		dropout=config["dropout"]
	)

### ResNet18

In [15]:
config = {
	"model_name": "resnet18", 
	"fc_dim": 512,
	"freeze_layers": False,
	"dropout": 0.5,
	"lr": 3e-4,
	"batch_size": 16
}

In [12]:
model = get_model(config).to(device)
criterion = ArcFaceLoss(CFG.image_size, CFG.n_classes, s=CFG.scale, m=CFG.margin, easy_margin=CFG.easy_margin).to(device)
optimizer = optim.Adam(model.parameters(), lr=config['lr'])
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min')

# Datasets and data loaders
training_sampler = torch.utils.data.SubsetRandomSampler(
    torch.arange(len(train_dataset))[:int(0.4*len(train_dataset))]
)
valid_sampler = torch.utils.data.SubsetRandomSampler(
    torch.arange(len(valid_dataset))[:int(0.4*len(valid_dataset))]
)
train_loader = torch.utils.data.DataLoader(
    train_dataset,
    sampler=training_sampler,
    batch_size=int(config["batch_size"]),
    # shuffle=True,
    num_workers=0,
)
valid_loader = torch.utils.data.DataLoader(
    valid_dataset,
    batch_size=int(config["batch_size"]),
    # shuffle=True,
    sampler=valid_sampler,
    num_workers=0,
)

train_losses = []
valid_losses = []
## Training loop: training + validation
for epoch in range(10):
    ## Training
    train_loss = 0.0
    epoch_steps = 0
    model.train()
    train_enumerator = enumerate(train_loader, 0)
    train_enumerator = tqdm(train_enumerator)
    for i, data in train_enumerator:
        input, target = data
        input, target = input.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(input)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        epoch_steps += 1
        if i % 50 == 0:
            print("[%d, %5d / %d batches] loss: %.3f" % (epoch + 1, i + 1, len(train_loader),
                                            train_loss / epoch_steps))
    

    ## Validation
    val_loss = 0.0
    model.eval()
    valid_enumerator = enumerate(valid_loader, 0)
    valid_enumerator = tqdm(valid_enumerator)
    for i, data in valid_enumerator:
        with torch.no_grad():
            input, target = data
            input, target = input.to(device), target.to(device)
            output = model(input)
            loss = criterion(output, target)
            val_loss += loss.cpu().numpy()

    train_losses.append(train_loss)
    valid_losses.append(val_loss)

RuntimeError: ignored

### DenseNet121

In [ ]:
config = {
	"model_name": "densenet121", 
	"fc_dim": train_df.label_group.nunique(),
	"freeze_layers": False,
	"dropout": 0.5,
	"lr": 3e-4,
	"batch_size": 16
}

In [ ]:
model = get_model(config).to(device)
criterion = ArcFaceLoss(CFG.image_size, train_df.label_group.nunique(), s=CFG.scale, m=CFG.margin, easy_margin=CFG.easy_margin).to(device)
optimizer = optim.Adam(model.parameters(), lr=config['lr'])
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min')

# Datasets and data loaders
training_sampler = torch.utils.data.SubsetRandomSampler(
    torch.arange(len(train_dataset))[:int(0.4*len(train_dataset))]
)
valid_sampler = torch.utils.data.SubsetRandomSampler(
    torch.arange(len(valid_dataset))[:int(0.4*len(valid_dataset))]
)
train_loader = torch.utils.data.DataLoader(
    train_dataset,
    sampler=training_sampler,
    batch_size=int(config["batch_size"]),
    # shuffle=True,
    num_workers=0,
)
valid_loader = torch.utils.data.DataLoader(
    valid_dataset,
    batch_size=int(config["batch_size"]),
    # shuffle=True,
    sampler=valid_sampler,
    num_workers=0,
)

train_losses = []
valid_losses = []
## Training loop: training + validation
for epoch in range(10):
    ## Training
    train_loss = 0.0
    epoch_steps = 0
    model.train()
    train_enumerator = enumerate(train_loader, 0)
    train_enumerator = tqdm(train_enumerator)
    for i, data in train_enumerator:
        input, target = data
        input, target = input.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(input)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        epoch_steps += 1
        if i % 50 == 0:
            print("[%d, %5d / %d batches] loss: %.3f" % (epoch + 1, i + 1, len(train_loader),
                                            train_loss / epoch_steps))
    

    ## Validation
    val_loss = 0.0
    model.eval()
    valid_enumerator = enumerate(valid_loader, 0)
    valid_enumerator = tqdm(valid_enumerator)
    for i, data in valid_enumerator:
        with torch.no_grad():
            input, target = data
            input, target = input.to(device), target.to(device)
            output = model(input)
            loss = criterion(output, target)
            val_loss += loss.cpu().numpy()

    train_losses.append(train_loss)
    valid_losses.append(val_loss)